#  IDK

In [1]:
!pip install tensorflow keras opencv-python


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0  # EfficientNetB0 is commonly used

# Load the EfficientNetB0 pre-trained on ImageNet (without the top classification layers)
backbone = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(None, None, 3))

# Check the model summary
backbone.summary()


2025-05-14 17:21:42.560934: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747243302.762899      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747243302.821368      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1747243317.674455      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "efficientnetb0"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None, None, 3)  │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, None, None, 3)  │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization             │ (None, None, None, 3)  │              7 │ rescaling[0][0]        │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_1 (Rescaling)   │ (None, None, None, 3)  │              0 │ normalization[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, None, None, 3)  │              0 │ rescaling_1[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, None, None, 32) │            864 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, None, None, 32) │            128 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, None, None, 32) │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, None, None, 32) │            288 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, None, None, 32) │            128 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, None, None, 32) │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 32)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 32)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 8)        │            264 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_expand    

 Total params: 4,049,571 (15.45 MB)

 Trainable params: 4,007,548 (15.29 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [3]:
# Freeze all layers except the last 5
for layer in backbone.layers[:-5]:
    layer.trainable = False

# Check if the layers are frozen
for layer in backbone.layers:
    print(f"{layer.name}: {layer.trainable}")


input_layer: False
rescaling: False
normalization: False
rescaling_1: False
stem_conv_pad: False
stem_conv: False
stem_bn: False
stem_activation: False
block1a_dwconv: False
block1a_bn: False
block1a_activation: False
block1a_se_squeeze: False
block1a_se_reshape: False
block1a_se_reduce: False
block1a_se_expand: False
block1a_se_excite: False
block1a_project_conv: False
block1a_project_bn: False
block2a_expand_conv: False
block2a_expand_bn: False
block2a_expand_activation: False
block2a_dwconv_pad: False
block2a_dwconv: False
block2a_bn: False
block2a_activation: False
block2a_se_squeeze: False
block2a_se_reshape: False
block2a_se_reduce: False
block2a_se_expand: False
block2a_se_excite: False
block2a_project_conv: False
block2a_project_bn: False
block2b_expand_conv: False
block2b_expand_bn: False
block2b_expand_activation: False
block2b_dwconv: False
block2b_bn: False
block2b_activation: False
block2b_se_squeeze: False
block2b_se_reshape: False
block2b_se_reduce: False
block2b_se_expa

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, Model

# Load EfficientNetB0 model with pre-trained ImageNet weights
base_model = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False)

# Freeze all layers except for the last 5 layers
for layer in base_model.layers:
    layer.trainable = False

for layer in base_model.layers[-5:]:
    layer.trainable = True



In [5]:
# Define the YOLO head
def yolo_head(base_model, grid_size, num_classes, num_bboxes):
    """
    YOLO head to predict bounding boxes and class probabilities.
    
    Arguments:
    - base_model: The pre-trained backbone (EfficientNet)
    - grid_size: Grid size (e.g., 13x13, 19x19)
    - num_classes: Number of classes in the dataset
    - num_bboxes: Number of bounding boxes to predict (usually 3 or 5)
    
    Returns:
    - The YOLO head layer that outputs bounding boxes and class probabilities
    """
    # Adding a convolutional layer to predict bounding boxes and class probabilities
    x = layers.Conv2D(filters=(grid_size * grid_size * (num_bboxes * 5 + num_classes)),
                      kernel_size=(1, 1), strides=(1, 1), padding="same")(base_model.output)

    # Reshape the output to match grid size, number of bounding boxes, and class predictions
    x = layers.Reshape(target_shape=(grid_size, grid_size, num_bboxes, 5 + num_classes))(x)
    
    return x



In [6]:
# Set parameters
grid_size = 13  # 13x13 grid for YOLOv3-like detection
num_classes = 20  # Example: 20 classes from VOC dataset
num_bboxes = 3  # Example: 3 bounding boxes per grid cell

# Add YOLO head to EfficientNet backbone
yolo_output = yolo_head(base_model, grid_size, num_classes, num_bboxes)

# Define the model
model = Model(inputs=base_model.input, outputs=yolo_output)

# Print model summary to confirm the architecture
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, None, None, 3)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_2 (Rescaling)   │ (None, None, None, 3)  │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization_1           │ (None, None, None, 3)  │              7 │ rescaling_2[0][0]      │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_3 (Rescaling)   │ (None, None, None, 3)  │              0 │ normalization_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, None, None, 3)  │              0 │ rescaling_3[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, None, None, 32) │            864 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, None, None, 32) │            128 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, None, None, 32) │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, None, None, 32) │            288 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, None, None, 32) │            128 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, None, None, 32) │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 32)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 32)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 8)        │            264 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 11,626,686 (44.35 MB)

 Trainable params: 8,358,555 (31.89 MB)

 Non-trainable params: 3,268,131 (12.47 MB)

In [7]:
import tensorflow as tf

def yolo_loss_voc(y_true, y_pred, lambda_coord=5.0, lambda_noobj=0.5):
    # Extract components
    true_box = y_true[..., 0:4]        # x, y, w, h
    true_obj = y_true[..., 4:5]        # object presence (0 or 1)
    true_class = y_true[..., 5:]       # one-hot class vector

    pred_box = y_pred[..., 0:4]
    pred_obj = y_pred[..., 4:5]
    pred_class = y_pred[..., 5:]

    # 1. Bounding box (x, y, w, h) loss — only for cells with objects
    coord_loss = lambda_coord * tf.reduce_sum(
        true_obj * tf.square(true_box - pred_box)
    )

    # 2. Objectness loss (for cells with objects)
    obj_loss = tf.reduce_sum(
        true_obj * tf.square(pred_obj - true_obj)
    )

    # 3. No-object loss (for cells without objects)
    noobj_loss = lambda_noobj * tf.reduce_sum(
        (1. - true_obj) * tf.square(pred_obj - true_obj)
    )

    # 4. Classification loss (cross-entropy) — only for cells with objects
    class_loss = tf.reduce_sum(
        true_obj * tf.keras.losses.categorical_crossentropy(true_class, pred_class)
    )

    # Total loss
    total_loss = coord_loss + obj_loss + noobj_loss + class_loss
    return total_loss


In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=yolo_loss_voc
)


In [9]:
train_dataset = load_dataset('/kaggle/input/pascal-voc-2012-tfrecord/train.record')
val_dataset = load_dataset('/kaggle/input/pascal-voc-2012-tfrecord/val.record')


NameError: name 'load_dataset' is not defined

In [ ]:
import os
import xml.etree.ElementTree as ET

def parse_annotations(annotations_dir, images_dir):
    annotations = []
    for xml_file in os.listdir(annotations_dir):
        if not xml_file.endswith('.xml'):
            continue
        tree = ET.parse(os.path.join(annotations_dir, xml_file))
        root = tree.getroot()
        filename = root.find('filename').text
        image_path = os.path.join(images_dir, filename)
        boxes = []
        labels = []
        for obj in root.findall('object'):
            label = obj.find('name').text
            bbox = obj.find('bndbox')
            box = [
                int(bbox.find('xmin').text),
                int(bbox.find('ymin').text),
                int(bbox.find('xmax').text),
                int(bbox.find('ymax').text)
            ]
            boxes.append(box)
            labels.append(label)
        annotations.append((image_path, boxes, labels))
    return annotations


# EfficientNet + Yolo 

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
import torch.nn.functional as F
import cv2
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from sklearn import preprocessing

# Define constants
BASE_PATH = "../input/pascal-voc-2012/VOC2012"
XML_PATH = os.path.join(BASE_PATH, "Annotations")
IMG_PATH = os.path.join(BASE_PATH, "JPEGImages")  # Your image directory
XML_FILES = [os.path.join(XML_PATH, f) for f in os.listdir(XML_PATH)]  # Path to your XML files
IMG_SIZE = 416  # YOLO standard image size
BATCH_SIZE = 4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# BASE_PATH = "../input/pascal-voc-2012/VOC2012"
# XML_PATH = os.path.join(BASE_PATH, "Annotations")
# IMG_PATH = os.path.join(BASE_PATH, "JPEGImages")
# XML_FILES = [os.path.join(XML_PATH, f) for f in os.listdir(XML_PATH)]

In [ ]:
# Define your class labels for Pascal VOC
classes = {1:'aeroplane', 2:'bicycle', 3:'bird', 4:'boat', 5:'bottle', 6:'bus', 7:'car', 8:'cat', 
           9:'chair', 10:'cow', 11:'diningtable', 12:'dog', 13:'horse', 14:'motorbike', 15:'person', 
           16:'pottedplant', 17:'sheep', 18:'sofa', 19:'train', 20:'tvmonitor'}


In [ ]:
class XmlParser(object):

    def __init__(self,xml_file):

        self.xml_file = xml_file
        self._root = ET.parse(self.xml_file).getroot()
        self._objects = self._root.findall("object")
        # path to the image file as describe in the xml file
        self.img_path = os.path.join(IMG_PATH, self._root.find('filename').text)
        # image id 
        self.image_id = self._root.find("filename").text
        # names of the classes contained in the xml file
        self.names = self._get_names()
        # coordinates of the bounding boxes
        self.boxes = self._get_bndbox()

    def parse_xml(self):
        """"Parse the xml file returning the root."""
    
        tree = ET.parse(self.xml_file)
        return tree.getroot()

    def _get_names(self):

        names = []
        for obj in self._objects:
            name = obj.find("name")
            names.append(name.text)

        return np.array(names)

    def _get_bndbox(self):

        boxes = []
        for obj in self._objects:
            coordinates = []
            bndbox = obj.find("bndbox")
            coordinates.append(np.int32(bndbox.find("xmin").text))
            coordinates.append(np.int32(np.float32(bndbox.find("ymin").text)))
            coordinates.append(np.int32(bndbox.find("xmax").text))
            coordinates.append(np.int32(bndbox.find("ymax").text))
            boxes.append(coordinates)

        return np.array(boxes)
def xml_files_to_df(xml_files):
    
    """"Return pandas dataframe from list of XML files."""
    
    names = []
    boxes = []
    image_id = []
    xml_path = []
    img_path = []
    for file in xml_files:
        xml = XmlParser(file)
        names.extend(xml.names)
        boxes.extend(xml.boxes)
        image_id.extend([xml.image_id] * len(xml.names))
        xml_path.extend([xml.xml_file] * len(xml.names))
        img_path.extend([xml.img_path] * len(xml.names))
    a = {"image_id": image_id,
         "names": names,
         "boxes": boxes,
         "xml_path":xml_path,
         "img_path":img_path}
    
    df = pd.DataFrame.from_dict(a, orient='index')
    df = df.transpose()
    
    return df

df = xml_files_to_df(XML_FILES)
df.head()

In [ ]:
# check values for per class
df['names'].value_counts()

In [ ]:
df['img_id'] = df['image_id'].apply(lambda x:x.split('.')).map(lambda x:x[0])
df.drop(columns=['image_id'], inplace=True)
df.head()

In [ ]:
# classes need to be in int form so we use LabelEncoder for this task
# enc = preprocessing.LabelEncoder()
# df['labels'] = enc.fit_transform(df['names'])
# df['labels'] = np.stack(df['labels'][i]+1 for i in range(len(df['labels']))) 
enc = preprocessing.LabelEncoder()
df['labels'] = enc.fit_transform(df['names']) + 1


In [ ]:
classes = df[['names','labels']].value_counts()
classes

In [ ]:
classes= {1:'aeroplane',2:'bicycle',3:'bird',4:'boat',5:'bottle',6:'bus',7:'car',8:'cat',9:'chair',10:'cow',11:'diningtable',12:'dog',13:'horse',14:'motorbike',15:'person',16:'pottedplant',17:'sheep',18:'sofa',19:'train',20:'tvmonitor'}

In [ ]:
# Convert each box into a list and stack them
df[['xmin', 'ymin', 'xmax', 'ymax']] = np.array([box for box in df['boxes']])

df.drop(columns=['boxes'], inplace=True)

# Convert coordinates to float
df['xmin'] = df['xmin'].astype(np.float64)
df['ymin'] = df['ymin'].astype(np.float64)
df['xmax'] = df['xmax'].astype(np.float64)
df['ymax'] = df['ymax'].astype(np.float64)


In [ ]:
# drop names column since we dont need it anymore
df.drop(columns=['names'], inplace=True)
df.head()

In [ ]:
len(df['img_id'].unique())

In [ ]:
image_ids = df['img_id'].unique()
valid_ids = image_ids[-4000:]
train_ids = image_ids[:-4000]
len(train_ids)

In [ ]:
valid_df = df[df['img_id'].isin(valid_ids)]
train_df = df[df['img_id'].isin(train_ids)]
valid_df.shape, train_df.shape

In [ ]:
train_df

In [ ]:
# Dataset class (no changes needed here, it works with YOLO and EfficientNet)
class VOCDataset(Dataset):
    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()
        self.image_ids = dataframe['img_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        records = self.df[self.df['img_id'] == image_id]

        image = cv2.imread(f'{self.image_dir}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        rows, cols = image.shape[:2]

        boxes = records[['xmin', 'ymin', 'xmax', 'ymax']].values
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)

        # label = records['labels'].values
        labels = records['labels'].values.tolist()  # Keep as plain list of ints


        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            # target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1,0)
            if sample['bboxes']:
                target['boxes'] = torch.tensor(sample['bboxes'], dtype=torch.float32)
            else:
                target['boxes'] = torch.zeros((0, 4), dtype=torch.float32)

        return image, target

    def __len__(self) -> int:
        return self.image_ids.shape[0]

# Transformations
def get_transform_train():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        ToTensorV2(p=1.0)
    ], bbox_params={'format':'pascal_voc', 'label_fields': ['labels']})

def get_transform_valid():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields':['labels']})



In [ ]:
# Create DataLoader
train_dataset = VOCDataset(train_df, IMG_PATH , get_transform_train())
valid_dataset = VOCDataset(valid_df, IMG_PATH, get_transform_valid())

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_data_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

# EfficientNet Backbone with YOLO Head (simplified)
# class EfficientNetYOLO(nn.Module):
#     def __init__(self, num_classes):
#         super(EfficientNetYOLO, self).__init__()
        
#         # Load pre-trained EfficientNet-B0 model
#         self.backbone = models.efficientnet_b0(pretrained=True)
        
#         # Replace classifier part for object detection
#         self.backbone.classifier = nn.Identity()
        
#         # Add YOLO head
#         self.yolo_head = nn.Sequential(
#             nn.Conv2d(1280, 512, kernel_size=1),  # YOLO detection head
#             nn.ReLU(),
#             nn.Conv2d(512, 256, kernel_size=3, padding=1),
#             nn.ReLU(),
#             nn.Conv2d(256, num_classes * 5, kernel_size=1)  # 5 values: [class, x_center, y_center, width, height]
#         )

#     def forward(self, x):
#         features = self.backbone(x)
#         detection = self.yolo_head(features)
#         return detection



In [ ]:
class EfficientNetYOLO(nn.Module):
    def __init__(self, num_classes, anchors=3):
        super(EfficientNetYOLO, self).__init__()

        # Load pre-trained EfficientNet-B0 model
        backbone = models.efficientnet_b0(pretrained=True)
        self.backbone = backbone.features  # Retain spatial dimensions

        # YOLO head - assuming output from EfficientNet has 1280 channels
        self.yolo_head = nn.Sequential(
            nn.Conv2d(1280, 512, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, num_classes * anchors * 5, kernel_size=1)  # Adjusted for anchor boxes
        )

    def forward(self, x):
        x = self.backbone(x)       # Output shape: [B, 1280, H/32, W/32]
        x = self.yolo_head(x)      # Output shape: [B, num_classes * 3 * 5, H/32, W/32]
        
        B, C, H, W = x.shape
        x = x.view(B, -1, 5, H, W)  # [B, num_classes, 5, H, W]
        x = x.permute(0, 3, 4, 1, 2)  # [B, H, W, num_classes, 5]
        return x


In [ ]:
# Define YOLO loss function
def yolo_loss(predictions, targets, num_classes, anchors, device):
    # Extract predictions
    pred_boxes = predictions[..., :4]  # [x, y, w, h]
    pred_classes = predictions[..., 4:]  # Class scores

    # Extract targets
    target_boxes = targets[..., :4]  # [x, y, w, h]
    target_classes = targets[..., 4].long()  # Class labels
    target_mask = targets[..., 4] > 0  # Mask for valid targets

    # Calculate IoU between predicted and ground truth boxes
    iou = calculate_iou(pred_boxes, target_boxes)

    # Define loss components
    object_mask = target_mask
    no_object_mask = 1 - object_mask

    # Box loss (IoU loss)
    box_loss = F.mse_loss(pred_boxes * object_mask, target_boxes * object_mask, reduction='sum')

    # Class loss (cross entropy)
    class_loss = F.cross_entropy(pred_classes.view(-1, num_classes), target_classes.view(-1), reduction='sum')

    # Object loss (confidence score for objectness)
    object_loss = F.binary_cross_entropy_with_logits(pred_classes[..., 0], object_mask, reduction='sum')

    # No object loss (confidence score for background)
    no_object_loss = F.binary_cross_entropy_with_logits(pred_classes[..., 0], no_object_mask, reduction='sum')

    # Total loss
    total_loss = box_loss + class_loss + object_loss + no_object_loss

    return total_loss

def calculate_iou(pred_boxes, target_boxes):
    # Calculate IoU (Intersection over Union) between predicted boxes and ground truth boxes
    pred_xmin, pred_ymin, pred_xmax, pred_ymax = pred_boxes[..., 0], pred_boxes[..., 1], pred_boxes[..., 2], pred_boxes[..., 3]
    target_xmin, target_ymin, target_xmax, target_ymax = target_boxes[..., 0], target_boxes[..., 1], target_boxes[..., 2], target_boxes[..., 3]

    inter_xmin = torch.max(pred_xmin, target_xmin)
    inter_ymin = torch.max(pred_ymin, target_ymin)
    inter_xmax = torch.min(pred_xmax, target_xmax)
    inter_ymax = torch.min(pred_ymax, target_ymax)

    inter_area = torch.max(inter_xmax - inter_xmin, torch.tensor(0.0).to(device)) * torch.max(inter_ymax - inter_ymin, torch.tensor(0.0).to(device))
    pred_area = (pred_xmax - pred_xmin) * (pred_ymax - pred_ymin)
    target_area = (target_xmax - target_xmin) * (target_ymax - target_ymin)

    union_area = pred_area + target_area - inter_area

    iou = inter_area / union_area
    return iou



In [ ]:
# # Model instantiation and moving to device
# model = EfficientNetYOLO(num_classes=20).to(DEVICE)

# # Training Loop (simplified)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# criterion = nn.CrossEntropyLoss()  # Choose your appropriate loss for YOLO (e.g., MSE for bounding boxes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EfficientNetYOLO(num_classes=20).to(device)

# Training setup
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# Training loop
def train_one_epoch(model, data_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0

    for images, targets in data_loader:
        # Move each image to device
        images = [img.to(device) for img in images]

        # Move each target dict to device
        targets = [
            {k: v.to(device) if torch.is_tensor(v) else v for k, v in t.items()}
            for t in targets
        ]

        # Forward pass
        outputs = model(torch.stack(images))  # If model expects batch tensor

        # Compute YOLO-style loss (custom)
        loss = yolo_loss(outputs, targets, num_classes=20, anchors=None, device=device)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        print(f"Batch Loss: {loss.item():.4f}")

    print(f"Epoch Loss: {total_loss / len(data_loader):.4f}")


In [ ]:
for epoch in range(10):  # Number of epochs
    train_one_epoch(model, train_data_loader, optimizer, yolo_loss, device)

In [ ]:
def train_one_epoch(model, data_loader, optimizer, criterion):
    model.train()
    total_loss = 0

    for images, targets in data_loader:
        # Move each image to device
        # images = [img.to(DEVICE) for img in images]
        images = torch.stack([img.to(DEVICE) for img in images])

        # Move each target dict to device
        targets = [
            {k: v.to(DEVICE) if torch.is_tensor(v) else v for k, v in t.items()}
            for t in targets
        ]

        # Forward pass
        outputs = model(torch.stack(images))  # If model expects batch tensor

        # Compute YOLO-style loss (custom)
        loss = criterion(targets, outputs)  # You must implement yolo_loss(targets, outputs)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        print(f"Batch Loss: {loss.item():.4f}")

    print(f"Epoch Loss: {total_loss / len(data_loader):.4f}")


In [ ]:
# Example for training loop
for epoch in range(10):  # Number of epochs
    train_one_epoch(model, train_data_loader, optimizer)

# ResNet + Yolo

In [ ]:
from transformers import YolosImageProcessor, YolosForObjectDetection
import torch
from PIL import Image
import requests

# Load model and processor
processor = YolosImageProcessor.from_pretrained("hustvl/yolos-base")
model = YolosForObjectDetection.from_pretrained("hustvl/yolos-base")

# Set to eval mode
model.eval()


In [ ]:
# Sample image
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

# Preprocess
inputs = processor(images=image, return_tensors="pt")

# Run inference
with torch.no_grad():
    outputs = model(**inputs)

# Postprocess
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

# Print results
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    print(f"{model.config.id2label[label.item()]}: {round(score.item(), 3)} - Box: {box.tolist()}")


In [ ]:
import os
dataset_path = '/kaggle/input/pascal-voc-2012-dataset/'
os.listdir(dataset_path)


In [ ]:
train_val_path = '/kaggle/input/pascal-voc-2012-dataset/VOC2012_train_val/VOC2012_train_val/'
os.listdir(train_val_path)


In [ ]:
import xml.etree.ElementTree as ET
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Sample image and annotation
image_id = '2007_006348'  # Replace with any image id from the dataset
image_path = os.path.join(train_val_path, 'JPEGImages', f'{image_id}.jpg')
annotation_path = os.path.join(train_val_path, 'Annotations', f'{image_id}.xml')

# Load image
img = Image.open(image_path)

# Parse XML file for annotation
tree = ET.parse(annotation_path)
root = tree.getroot()

# Plot the image
fig, ax = plt.subplots(1)
ax.imshow(img)

# Draw bounding boxes
for obj in root.findall('object'):
    name = obj.find('name').text
    bndbox = obj.find('bndbox')
    xmin = int(bndbox.find('xmin').text)
    ymin = int(bndbox.find('ymin').text)
    xmax = int(bndbox.find('xmax').text)
    ymax = int(bndbox.find('ymax').text)

    # Create a Rectangle patch for the bounding box
    rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, linewidth=2, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

    # Add label for the object
    plt.text(xmin, ymin, name, color='white', fontsize=12, verticalalignment='bottom')

# Show the image with bounding boxes
plt.axis('off')
plt.show()


In [ ]:
from torchvision import transforms

# Define the transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to fixed size
    transforms.ToTensor(),  # Convert image to tensor
])

# Create the dataset and DataLoader
train_dataset = VOCDataset(root='./VOCdevkit', image_set='train', transforms=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# Check dataset output
for images, targets in train_loader:
    print(images[0].shape)  # Print shape of one image
    print(targets[0])  # Print targets (bounding boxes and labels)
    break


In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import xml.etree.ElementTree as ET
from PIL import Image
from torch.nn.utils.rnn import pad_sequence  # Import padding function

# Class label mapping for Pascal VOC
LABEL_MAP = {
    'aeroplane': 1, 'bicycle': 2, 'bird': 3, 'boat': 4, 'bottle': 5, 'bus': 6,
    'car': 7, 'cat': 8, 'chair': 9, 'cow': 10, 'diningtable': 11, 'dog': 12,
    'horse': 13, 'motorbike': 14, 'person': 15, 'pottedplant': 16, 'sheep': 17,
    'sofa': 18, 'train': 19, 'tvmonitor': 20
}

# Dataset Class for Pascal VOC 2012
class VOCDataset(Dataset):
    def __init__(self, root, image_set='train', transforms=None):
        self.root = root
        self.image_set = image_set
        self.transforms = transforms
        self.image_dir = os.path.join(root, 'JPEGImages')  # Image directory
        self.annotation_dir = os.path.join(root, 'Annotations')  # Annotations directory
        image_set_file = os.path.join(root, 'ImageSets', 'Main', f'{image_set}.txt')  # Image set file

        with open(image_set_file, 'r') as f:
            self.image_ids = [x.strip() for x in f.readlines()]

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        image_path = os.path.join(self.image_dir, f'{image_id}.jpg')  # Image path
        annotation_path = os.path.join(self.annotation_dir, f'{image_id}.xml')  # Annotation path

        img = Image.open(image_path).convert("RGB")

        # Parse XML annotation
        tree = ET.parse(annotation_path)
        root = tree.getroot()

        boxes = []
        labels = []

        for obj in root.findall('object'):
            name = obj.find('name').text
            if name not in LABEL_MAP:
                continue
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)

            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(LABEL_MAP[name])  # Convert label to integer

        # Convert to tensors
        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        # Apply any transformations
        if self.transforms:
            img = self.transforms(img)

        target = {
            "boxes": boxes,
            "labels": labels,
        }

        return img, target

# Define a transformation pipeline
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Define collate_fn to handle different numbers of bounding boxes in each image
def collate_fn(batch):
    images = []
    targets = []
    for img, target in batch:
        images.append(img)
        targets.append(target)

    images = torch.stack(images, dim=0)

    # Pad bounding boxes (if necessary) to match the maximum number of boxes in the batch
    boxes = [target['boxes'] for target in targets]
    labels = [target['labels'] for target in targets]

    # Pad bounding boxes to have the same number of elements (max number of boxes)
    padded_boxes = pad_sequence(boxes, batch_first=True, padding_value=-1)  # Padding value for empty boxes
    padded_labels = pad_sequence(labels, batch_first=True, padding_value=-1)  # Padding value for empty labels

    for target, padded_box, padded_label in zip(targets, padded_boxes, padded_labels):
        target['boxes'] = padded_box
        target['labels'] = padded_label

    return images, targets

# Define the dataset and dataloader
train_dataset = VOCDataset(root='/kaggle/input/pascal-voc-2012-dataset/VOC2012_train_val/VOC2012_train_val', image_set='train', transforms=transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

# Test dataset loading (checking images and bounding boxes)
for images, targets in train_loader:
    print(images[0].shape)  # Image shape
    print(targets[0]['boxes'])  # Bounding box
    print(targets[0]['labels'])  # Label
    break  # Just checking for the first batch


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualize(img, boxes, labels):
    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(img)

    for box, label in zip(boxes, labels):
        if label != -1:  # Ignore padding labels
            xmin, ymin, xmax, ymax = box
            width = xmax - xmin
            height = ymax - ymin
            rect = patches.Rectangle((xmin, ymin), width, height, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            ax.text(xmin, ymin, f'Label: {label}', fontsize=12, color='red')

    plt.show()

# Visualize the first image and its bounding boxes
img, target = train_dataset[0]
visualize(img.permute(1, 2, 0), target['boxes'], target['labels'])


In [ ]:
# import torch
# import torch.nn as nn
# import torchvision.models as models

# class ResNetYOLO(nn.Module):
#     def __init__(self, num_classes=21):  # Pascal VOC has 21 classes (including background)
#         super(ResNetYOLO, self).__init__()

#         # Load a pre-trained ResNet50 backbone
#         self.backbone = models.resnet50(pretrained=True)

#         # Remove the fully connected layers (classification head)
#         self.backbone = nn.Sequential(*list(self.backbone.children())[:-2])

#         # Freeze the backbone
#         for param in self.backbone.parameters():
#             param.requires_grad = False

#         # Adding a pooling layer to reduce feature map size before the fully connected layers
#         self.pool = nn.AdaptiveAvgPool2d((1, 1))

#         # Fully connected layers for bounding box and class prediction
#         # self.fc_bbox = nn.Linear(2048, 4)  # 4 for bounding box coordinates (x1, y1, x2, y2)
#         self.fc_bbox = nn.Linear(2048, 2 * 4)
#   # 2 boxes × 4 coords

#         self.fc_cls = nn.Linear(2048, num_classes)  # Output class probabilities
#     def forward(self, x):
#         batch_size = x.size(0)
#         num_boxes = 2  # Fixed: predicting 2 boxes per image
#         num_classes = self.fc_cls.out_features  # Dynamically get number of output classes
    
#         # Forward pass through the backbone
#         features = self.backbone(x)
    
#         # Apply global average pooling
#         pooled_features = self.pool(features)
    
#         # Flatten the features
#         pooled_features = pooled_features.view(batch_size, -1)
    
#         # Predict bounding boxes and class scores
#         bbox_preds = self.fc_bbox(pooled_features).view(batch_size, num_boxes, 4)  # (B, 2, 4)
#         class_preds = self.fc_cls(pooled_features).view(batch_size, num_boxes, num_classes)  # (B, 2, 21)
    
#         return bbox_preds, class_preds

# # Instantiate the model
# model = ResNetYOLO(num_classes=21)  # Pascal VOC has 21 classes

# # Move model to GPU (if available)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # Print the model architecture to verify
# print(model)
import torch
import torch.nn as nn
import torchvision.models as models

class ResNetYOLO(nn.Module):
    def __init__(self, num_classes=21):  # Pascal VOC has 21 classes (including background)
        super(ResNetYOLO, self).__init__()

        # Load a pre-trained ResNet50 backbone
        self.backbone = models.resnet50(pretrained=True)

        # Remove the fully connected layers (classification head)
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-2])

        # Freeze the backbone
        for param in self.backbone.parameters():
            param.requires_grad = False

        # Adding a pooling layer to reduce feature map size before the fully connected layers
        self.pool = nn.AdaptiveAvgPool2d((1, 1))

        # Fully connected layers for bounding box and class prediction
        # self.fc_bbox = nn.Linear(2048, 4 * 4)  # 2 boxes × 4 coordinates (x1, y1, x2, y2)
        self.fc_bbox = nn.Linear(2048, 2 * 4)  # 2 boxes × 4 coordinates (x1, y1, x2, y2)

        # Adjusting fc_cls to output (2 * num_classes), since there are 2 bounding boxes per image
        self.fc_cls = nn.Linear(2048, 2 * num_classes)  # 2 boxes × num_classes
        # self.fc_bbox = nn.Linear(2048, 4 * 4)  # Now predict 4 boxes × 4 coordinates

    def forward(self, x):
        batch_size = x.size(0)
        num_boxes = 2  # Fixed: predicting 2 boxes per image
        num_classes = self.fc_cls.out_features // num_boxes  # Dynamic number of classes

        # Forward pass through the backbone
        features = self.backbone(x)

        # Apply global average pooling to reduce spatial dimensions to (1, 1)
        pooled_features = self.pool(features)

        # Flatten the features
        pooled_features = pooled_features.view(batch_size, -1)

        # Predict bounding boxes and class scores
        # bbox_preds = self.fc_bbox(pooled_features).view(batch_size, 4, 4)  # 4 boxes × 4 coords

        bbox_preds = self.fc_bbox(pooled_features).view(batch_size, num_boxes, 4)  # (B, 2, 4)
        class_preds = self.fc_cls(pooled_features).view(batch_size, num_boxes, num_classes)  # (B, 2, 21)

        return bbox_preds, class_preds

# Instantiate the model
model = ResNetYOLO(num_classes=21)  # Pascal VOC has 21 classes

# Move model to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Print the model architecture to verify
print(model)


In [ ]:
import torch
import torch.nn.functional as F

# def yolo_loss(pred_boxes, pred_classes, true_boxes, true_classes, lambda_coord=5, lambda_noobj=0.5):
#     """
#     Calculate YOLO loss function.
#     Args:
#         pred_boxes: Predicted bounding box coordinates (batch_size, num_boxes, 4)
#         pred_classes: Predicted class probabilities (batch_size, num_boxes, num_classes)
#         true_boxes: Ground truth bounding box coordinates (batch_size, num_boxes, 4)
#         true_classes: Ground truth class labels (batch_size, num_boxes)
#         lambda_coord: Weighting factor for bounding box loss
#         lambda_noobj: Weighting factor for no-object loss
#     """
#     # Bounding box loss (MSE)
#     box_loss = F.mse_loss(pred_boxes, true_boxes, reduction='sum')

#     # Class loss (cross-entropy)
#     class_loss = F.cross_entropy(pred_classes.view(-1, pred_classes.size(-1)), true_classes.view(-1), reduction='sum')

#     # Calculate objectness loss (binary cross-entropy)
#     obj_loss = F.binary_cross_entropy_with_logits(pred_classes[..., 0], true_classes.float(), reduction='sum')

#     # No-objectness loss (binary cross-entropy)
#     noobj_loss = F.binary_cross_entropy_with_logits(pred_classes[..., 1:], torch.zeros_like(pred_classes[..., 1:]), reduction='sum')

#     # Total loss
#     total_loss = lambda_coord * box_loss + class_loss + lambda_noobj * noobj_loss + obj_loss

#     return total_loss
def yolo_loss(pred_boxes, pred_classes, true_boxes, true_classes, lambda_coord=5, lambda_noobj=0.5):
    """
    Calculate YOLO loss function.
    Args:
        pred_boxes: Predicted bounding box coordinates (batch_size, num_boxes, 4)
        pred_classes: Predicted class probabilities (batch_size, num_boxes, num_classes)
        true_boxes: Ground truth bounding box coordinates (batch_size, num_boxes, 4)
        true_classes: Ground truth class labels (batch_size, num_boxes)
        lambda_coord: Weighting factor for bounding box loss
        lambda_noobj: Weighting factor for no-object loss
    """
    # Bounding box loss (MSE)
    box_loss = F.mse_loss(pred_boxes, true_boxes, reduction='sum')

    # Class loss (cross-entropy)
    class_loss = F.cross_entropy(pred_classes.reshape(-1, pred_classes.size(-1)), true_classes.reshape(-1), reduction='sum')

    # Calculate objectness loss (binary cross-entropy)
    obj_loss = F.binary_cross_entropy_with_logits(pred_classes[..., 0], true_classes.float(), reduction='sum')

    # No-objectness loss (binary cross-entropy)
    noobj_loss = F.binary_cross_entropy_with_logits(pred_classes[..., 1:], torch.zeros_like(pred_classes[..., 1:]), reduction='sum')

    # Total loss
    total_loss = lambda_coord * box_loss + class_loss + lambda_noobj * noobj_loss + obj_loss

    return total_loss



In [ ]:
print("pred_boxes shape:", pred_boxes.shape)
print("pred_classes shape:", pred_classes.shape)
print("true_boxes shape:", true_boxes.shape)
print("true_classes shape:", true_classes.shape)


In [ ]:
import torch
import torch.optim as optim

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, targets in train_loader:
        images = images.to(device)

        # Process targets
        # Assuming the targets are dictionaries with 'boxes' and 'labels'
        target_boxes = torch.stack([target['boxes'] for target in targets]).to(device)   # Shape: (B, 2, 4)
        target_classes = torch.stack([target['labels'] for target in targets]).to(device)  # Shape: (B, 2)

        # Optional: If the number of boxes in ground truth differs from prediction, adjust here
        # Example: Keep only the first 2 boxes
        target_boxes = target_boxes[:, :2, :]  # Keep only the first 2 boxes
        target_classes = target_classes[:, :2]  # Keep only the first 2 classes

        optimizer.zero_grad()

        # Forward pass
        pred_boxes, pred_classes = model(images)  # pred_boxes: (B, 2, 4), pred_classes: (B, 2, num_classes)

        # Compute loss
        loss = yolo_loss(pred_boxes, pred_classes, target_boxes, target_classes)

        # Backpropagation
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")


In [ ]:
device = torch.device("cpu")  # Temporarily switch to CPU for debugging
model.to(device)


# Not Needed 

In [ ]:
images, targets = next(iter(train_loader))
images = images.to(device)

# Forward pass
pred_boxes, pred_classes = model(images)

print("pred_boxes:", pred_boxes.shape)     # Should be (batch_size, 8)
print("pred_classes:", pred_classes.shape) # Should be (batch_size, num_classes)


In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset
import os
import xml.etree.ElementTree as ET
from PIL import Image

class VOCDataset(Dataset):
    def __init__(self, root, image_set='train', transforms=None):
        """
        Args:
            root (string): Root directory of the VOC dataset.
            image_set (string): 'train', 'test', or 'val'.
            transforms (callable, optional): Optional transform to be applied on a sample.
        """
        self.root = root
        self.image_set = image_set
        self.transforms = transforms
        self.image_dir = os.path.join(root, 'VOC2012', 'JPEGImages')
        self.annotation_dir = os.path.join(root, 'VOC2012', 'Annotations')

        # Load image ids based on the split (train/test/val)
        image_set_file = os.path.join(root, 'VOC2012', 'ImageSets', 'Main', f'{image_set}.txt')
        with open(image_set_file, 'r') as f:
            self.image_ids = [x.strip() for x in f.readlines()]

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        # Get image id
        image_id = self.image_ids[idx]
        image_path = os.path.join(self.image_dir, f'{image_id}.jpg')
        annotation_path = os.path.join(self.annotation_dir, f'{image_id}.xml')

        # Load image
        img = Image.open(image_path).convert("RGB")

        # Parse the XML annotation
        tree = ET.parse(annotation_path)
        root = tree.getroot()

        boxes = []
        labels = []

        # Extract bounding boxes and labels
        for obj in root.findall('object'):
            name = obj.find('name').text
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(name)

        # Convert to tensors
        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels)

        # Apply any transformations
        if self.transforms:
            img = self.transforms(img)

        target = {
            "boxes": boxes,
            "labels": labels,
        }

        return img, target


In [ ]:
# # Extract only 2 boxes per image (as your model outputs 2 boxes)
# true_boxes = torch.stack([t['boxes'][:2] for t in targets])  # (batch_size, 2, 4)
# true_boxes = true_boxes.view(true_boxes.size(0), -1).to(device)  # (batch_size, 8)

# # Pad or trim to 2 labels
# true_classes = torch.stack([F.pad(t['labels'][:2], (0, 2 - len(t['labels'][:2])), value=-1) for t in targets]).to(device)

# Ground truth boxes: shape should be (batch_size, 2, 4)
true_boxes = torch.stack([F.pad(t['boxes'], (0, 0, 0, max(0, 2 - t['boxes'].shape[0])), value=0)[:2] for t in targets]).to(device)

# Ground truth class: assume the most confident class per image (use majority class or first label)
true_classes = torch.tensor([t['labels'][0] if len(t['labels']) > 0 else 0 for t in targets]).to(device)


In [ ]:
loss = yolo_loss(pred_boxes, pred_classes, true_boxes, true_classes)


In [ ]:
loss

In [ ]:
loss.backward()
optimizer.step()
running_loss += loss.item()


In [ ]:
import torch
import torch.optim as optim

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, targets in train_loader:
        images = images.to(device)

        # Process targets
        target_boxes = torch.stack([target['boxes'] for target in targets]).to(device)   # Shape: (B, 2, 4)
        target_classes = torch.stack([target['labels'] for target in targets]).to(device)  # Shape: (B,)

        optimizer.zero_grad()

        # Forward pass
        pred_boxes, pred_classes = model(images)  # pred_boxes: (B, 2, 4), pred_classes: (B, num_classes)

        # Compute loss
        loss = yolo_loss(pred_boxes, pred_classes, target_boxes, target_classes)

        # Backpropagation
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")


In [ ]:
print("pred_boxes shape:", pred_boxes.shape)
print("true_boxes shape:", true_boxes.shape)


In [ ]:
print("pred_classes shape:", pred_classes.shape)
print("true_classes shape:", true_classes.shape)


In [ ]:
for images, targets in train_loader:
    print("Images shape:", images.shape)
    break  # Check the first batch only


In [ ]:
for images, targets in train_loader:
    print("Targets format:", targets)
    # Print shapes of boxes and labels for each target
    for target in targets:
        print("Boxes shape:", target['boxes'].shape)
        print("Labels shape:", target['labels'].shape)
    break  # Check the first batch only


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualize_image_with_boxes(image, target, num_classes=20):
    fig, ax = plt.subplots(1, figsize=(10, 10))
    image = image.permute(1, 2, 0).cpu().numpy()
    image = (image * 255).astype("uint8")
    ax.imshow(image)

    VOC_CLASSES = [
        'aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
        'bus', 'car', 'cat', 'chair', 'cow', 'diningtable',
        'dog', 'horse', 'motorbike', 'person', 'pottedplant',
        'sheep', 'sofa', 'train', 'tvmonitor'
    ]

    # Filter out invalid boxes
    valid = target['labels'] >= 0
    boxes = target['boxes'][valid]
    labels = target['labels'][valid]

    for box, label in zip(boxes, labels):
        rect = patches.Rectangle(
            (box[0], box[1]),
            box[2] - box[0],
            box[3] - box[1],
            linewidth=2,
            edgecolor='r',
            facecolor='none'
        )
        ax.add_patch(rect)
        class_name = VOC_CLASSES[label.item()] if label.item() < len(VOC_CLASSES) else f"Unknown ({label.item()})"
        ax.text(box[0], box[1] - 5, class_name, color='white', fontsize=12, bbox=dict(facecolor='red', alpha=0.5))

    plt.axis("off")
    plt.show()
for images, targets in train_loader:
    visualize_image_with_boxes(images[0], targets[0])
    break


In [ ]:
# Get first image tensor from the batch
image_tensor = images[0]

print("Tensor dtype:", image_tensor.dtype)
print("Min pixel value:", image_tensor.min())
print("Max pixel value:", image_tensor.max())


In [ ]:
valid_mask = target['labels'] >= 0
boxes = target['boxes'][valid_mask]
labels = target['labels'][valid_mask]
# Should be one of 0–19
print("Labels in batch:", target['labels'])


In [ ]:
import os
import torch
from torch.utils.data import Dataset
import xml.etree.ElementTree as ET
from PIL import Image

class VOCDataset(Dataset):
    def __init__(self, root, image_set='train', transforms=None):
        """
        Args:
            root (string): Root directory of the VOC dataset.
            image_set (string): 'train', 'test', or 'val'.
            transforms (callable, optional): Optional transform to be applied on a sample.
        """
        self.root = root
        self.image_set = image_set
        self.transforms = transforms
        self.image_dir = os.path.join(root, 'JPEGImages')  # Updated image directory
        self.annotation_dir = os.path.join(root, 'Annotations')  # Updated annotations directory

        # Load image ids based on the split (train/test/val)
        image_set_file = os.path.join(root, 'ImageSets', 'Main', f'{image_set}.txt')  # Updated image set file path
        with open(image_set_file, 'r') as f:
            self.image_ids = [x.strip() for x in f.readlines()]

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        # Get image id
        image_id = self.image_ids[idx]
        image_path = os.path.join(self.image_dir, f'{image_id}.jpg')  # Updated image path
        annotation_path = os.path.join(self.annotation_dir, f'{image_id}.xml')  # Updated annotation path

        # Load image
        img = Image.open(image_path).convert("RGB")

        # Parse the XML annotation
        tree = ET.parse(annotation_path)
        root = tree.getroot()

        boxes = []
        labels = []

        # Extract bounding boxes and labels
        for obj in root.findall('object'):
            name = obj.find('name').text
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(name)

        # Convert to tensors
        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels)

        # Apply any transformations
        if self.transforms:
            img = self.transforms(img)

        target = {
            "boxes": boxes,
            "labels": labels,
        }

        return img, target
